In [1]:
import torch
import torch.nn.functional as F
import numpy as np

### Simple functions

$$ y = f(x) = \sum{(x^2 + 2 \cdot x)} $$

In [2]:
x = torch.arange(10, dtype=torch.float, requires_grad=True)

y = torch.sum(x ** 2 + 2 * x)

In [3]:
dy_dx_analytic = 2 * x + 2 

In [4]:
y.backward(retain_graph=True)  # calculates gradient w.r.t. graph nodes

In [5]:
dy_dx_numeric = x.grad.clone()

In [6]:
bool(torch.all(dy_dx_numeric == dy_dx_analytic))

True

$$ y = W_{hy} h $$
$$ p = softmax(y) $$
$$ loss = -log(p) $$

In [7]:
n = 10
m = 20

w = torch.randn(n, m, requires_grad=True)
h = torch.randint(3, (20, 1), dtype=torch.float)
y = torch.matmul(w, h)
p = F.softmax(y, dim=0)

label = torch.zeros_like(p)
label[5] = 1.

loss = -torch.sum(label * torch.log(p))

In [8]:
loss

tensor(6.4213, grad_fn=<NegBackward>)

In [9]:
loss.backward()

In [10]:
w_analytic_grad = torch.matmul((p - label) , h.view(1, -1))

In [11]:
torch.allclose(w_analytic_grad,  w.grad.data)

True

In [12]:
class SimpleNNNumpy:
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        self.w_1 = np.random.randn(hidden_size, input_size)
        self.w_2 = np.random.randn(output_size, hidden_size)
        
        self.cache = {}
    
    def forward(self, x: torch.tensor):
        h_1 = np.dot(self.w_1, x)
        z_1 = sigmoid(h_1)
        h_2 = np.dot(self.w_2, z_1)
        z_2 = stable_softmax(h_2)
        
        self.cache['z_1'] = z_1
        self.cache['z_2'] = z_2
        return z_2
    
    def loss(self, x: torch.tensor, label: torch.tensor):
        pred = self.forward(x)
        return -np.sum(label * np.log(pred))
    
    def backward(self, x: torch.tensor, label: torch.tensor):
        self.forward(x)
        
        z_1, z_2 = self.cache['z_1'], self.cache['z_2']

        dh_2 = z_2 - label
        dw_2 = np.dot(dh_2, z_1.T)
        dh_1 = np.dot(self.w_2.T, dh_2) * (z_1 * (1 - z_1))
        dw_1 = np.dot(dh_1, x.T)
        return dw_1, dw_2 
    
    
    def numerical_gradients(self, x: torch.tensor, label: torch.tensor, epsilon: float):
        d_params = (np.zeros_like(self.w_1)*1., np.zeros_like(self.w_2)*1.)
        params = (self.w_1, self.w_2)

        # calculating numerical gradients for each parameter
        for d_param, param in zip(d_params, params):

            # iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index

                # keeping the original value so we can reset it later
                original_value = param[ix]

                # estimating numeric gradients

                # x + epsilon
                param[ix] = original_value + epsilon
                loss_plus = self.loss(x, label)

                # x - epsilon
                param[ix] = original_value - epsilon
                loss_minus = self.loss(x, label)

                # numeric_gradient = (f(x + delta) - f(x - delta)) / (2 * delta)
                d_param[ix] = (loss_plus - loss_minus) / (2 * epsilon)

                # resetting parameter to original value
                param[ix] = original_value

                it.iternext()

        return d_params

        
    def gradient_check(self, 
                       x: torch.tensor,
                       label: torch.tensor,
                       epsilon: float = 1e-3,
                       threshold: float = 1e-5):
        """
        Performs gradient checking for model parameters:
         - computes the analytic gradients using our back-propagation implementation
         - computes the numerical gradients using the two-sided epsilon method
         - computes the relative difference between numerical and analytical gradients
         - checks that the relative difference is less than threshold
         - if the last check is failed, then raises an error
        """
        
        def check_relative_difference(a: torch.tensor, b: torch.tensor, threshold: float) -> bool:
            """Returns True if (|a - b| / (|a| + |b|)) > threshold else False."""
            print()
            numeratore = np.abs(a - b)
            print(np.max(numeratore))
            denominatore = np.abs(a) + np.abs(b)
            result = numeratore / denominatore
            result[np.isnan(result)] = 0
            print(np.max(result))
            return np.any(result > threshold)
        
        
        params = ('w_1', 'w_2')

        # calculating the gradients using backpropagation, aka analytic gradients
        self.cache = {}
        analytic_gradients = self.backward(x, label)
#         analytic_gradients = (x.clone() for x in analytic_gradients)

        # calculating numerical gradients
        self.cache = {}
        numeric_gradients = self.numerical_gradients(x, label, epsilon)

        # gradient check for each parameter
        for p_name, d_analytic, d_numeric in zip(params, analytic_gradients, numeric_gradients):
            print(f"\nPerforming gradient check for parameter {p_name} "
                  f"with size = {np.prod(d_analytic.shape)}.")
            
            if (not d_analytic.shape == d_numeric.shape or
                    check_relative_difference(d_analytic, d_numeric, threshold)):
               
                raise ValueError(f'Gradient check for {p_name} is failed.')

            print(f"Gradient check for parameter {p_name} is passed.")
        

### Simple NN

**Forward pass:**

$$ 
h_1 = w_1 \cdot x \\
z_1 = \sigma(h_1)  \\ 
h_2 = w_2 \cdot z_1 \\
z_2 = softmax (h_2)
$$

**Loss - Cross Entropy:**

$$ J = -label \cdot \log(z_2) $$

**Backward pass:**

$$
\frac {\partial J} {\partial w_2} = 
\frac {\partial J} {\partial h_2} 
\frac {\partial h_2} {\partial w_2} = 
(z_2 - label) \cdot z_1^T
$$

$$
\frac {\partial J} {\partial w_1} = 
\frac {\partial J} {\partial h_1} 
\frac {\partial h_1} {\partial w_1} = 
\frac {\partial J} {\partial h_1}  \cdot x^T
$$


$$
\frac {\partial J} {\partial h_1} = 
\frac {\partial J} {\partial h_2} 
\frac {\partial h_2} {\partial z_1}
\frac {\partial z_1} {\partial h_1}
= 
\big (w_1^T \cdot (z_2 - label) \big) z_1(1 - z_1)
$$

Here is the implementation of 2 layer simple neural network, which does the backpropogation mannually.

In [13]:
def check_relative_difference(a: torch.tensor, b: torch.tensor, threshold: float) -> bool:
    """Returns True if (|a - b| / (|a| + |b|)) > threshold else False."""
    numeratore = torch.abs(a - b)
    denominatore = torch.abs(a) + torch.abs(b)
    result = numeratore / denominatore
    result[torch.isnan(result)] = 0
    return bool(torch.any(result > threshold))

In [62]:
class SimpleNN:
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        self.w_1 = torch.randn(hidden_size, input_size, dtype=torch.float64) * 0.01
        self.w_2 = torch.randn(output_size, hidden_size, dtype=torch.float64) * 0.01
        
        self.cache = {}
    
    def forward(self, x: torch.tensor):
        h_1 = torch.matmul(self.w_1, x)
        z_1 = torch.sigmoid(h_1)
        h_2 = torch.matmul(self.w_2, z_1)
        z_2 = F.log_softmax(h_2, dim=0)
        
        self.cache['z_1'] = z_1
        self.cache['z_2'] = z_2
        return z_2
    
    def loss(self, x: torch.tensor, label: torch.tensor):
        log_pred = self.forward(x)
        return -torch.sum(label * log_pred)
    
    def backward(self, x: torch.tensor, label: torch.tensor):
        self.forward(x)
        
        z_1, z_2 = self.cache['z_1'], self.cache['z_2']

        dh_2 = torch.exp(z_2) - label
        dw_2 = torch.matmul(dh_2, z_1.t())
        dh_1 = torch.matmul(self.w_2.t(), dh_2) * (z_1 * (1 - z_1))
        dw_1 = torch.matmul(dh_1, x.t())
        return dw_1, dw_2
    
    def sgd_step(self, x: torch.tensor, label: torch.tensor, lr: float):
        dw_1, dw_2 = self.backward(x, label)
        self.w_1 -= lr * dw_1
        self.w_2 -= lr * dw_2
    
    def numerical_gradients(self, x: torch.tensor, label: torch.tensor, epsilon: float):
        d_params = (
            torch.zeros_like(self.w_1, dtype=torch.float64),  
            torch.zeros_like(self.w_2, dtype=torch.float64)
        )
        params = (self.w_1, self.w_2)

        # calculating numerical gradients for each parameter
        for d_param, param in zip(d_params, params):

            # iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index

                # keeping the original value so we can reset it later
                original_value = param[ix].item()

                # estimating numeric gradients

                # x + epsilon
                param[ix] = original_value + epsilon
                loss_plus = self.loss(x, label)

                # x - epsilon
                param[ix] = original_value - epsilon
                loss_minus = self.loss(x, label)

                # numeric_gradient = (f(x + delta) - f(x - delta)) / (2 * delta)
                d_param[ix] = ((loss_plus - loss_minus) / (2 * epsilon)).item()

                # resetting parameter to original value
                param[ix] = original_value
                it.iternext()

        return d_params
    
    def gradient_check(self, 
                       x: torch.tensor,
                       label: torch.tensor,
                       epsilon: float = 1e-1,
                       threshold: float = 1e-5):
        """
        Performs gradient checking for model parameters:
         - computes the analytic gradients using our back-propagation implementation
         - computes the numerical gradients using the two-sided epsilon method
         - computes the relative difference between numerical and analytical gradients
         - checks that the relative difference is less than threshold
         - if the last check is failed, then raises an error
        """
        params = ('w_1', 'w_2')

        # calculating the gradients using backpropagation, aka analytic gradients
        self.cache = {}
        analytic_gradients = self.backward(x, label)

        # calculating numerical gradients
        self.cache = {}
        numeric_gradients = self.numerical_gradients(x, label, epsilon)

        # gradient check for each parameter
        for p_name, d_analytic, d_numeric in zip(params, analytic_gradients, numeric_gradients):
            print(f"\nPerforming gradient check for parameter {p_name} "
                  f"with size = {np.prod(d_analytic.shape)}.")
            
            if (not d_analytic.shape == d_numeric.shape or
                    check_relative_difference(d_analytic, d_numeric, threshold)):
                            
                raise ValueError(f'Gradient check for {p_name} is failed.')

            print(f"Gradient check for parameter {p_name} is passed.")
            
    
            
        

In [63]:
print('Testing implementation.')

nn = SimpleNN(10, 20, 3)

x = torch.arange(10, dtype=torch.float64).view(10, 1)
label = torch.tensor([0, 0, 1.], dtype=torch.float64).reshape(3, 1)

log_pred = nn.forward(x)
pred = torch.exp(log_pred)

assert pred.shape == label.shape == (3, 1)
assert bool(abs(sum(pred) - 1.) < 1e-6)

loss = nn.loss(x, label)
assert torch.equal(loss, -log_pred[2, 0])

dw_1, dw_2 = nn.backward(x, label)
assert dw_1.shape == nn.w_1.shape == (20, 10)
assert dw_2.shape == nn.w_2.shape == (3, 20)

print('\nShapes are correct.')

nn.gradient_check(x, label, epsilon=1e-3, threshold=1e-4)

Testing implementation.

Shapes are correct.

Performing gradient check for parameter w_1 with size = 200.
Gradient check for parameter w_1 is passed.

Performing gradient check for parameter w_2 with size = 60.
Gradient check for parameter w_2 is passed.


Now lets apply the network to simple data set.

In [71]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


data = load_breast_cancer()
X = data['data']
label = data['target'] 

print(f'Data shape: {X.shape}')
print(f'Targets shape: {label.shape}')

Data shape: (569, 30)
Targets shape: (569,)


In [72]:
X = X.reshape(X.shape[0], X.shape[1], 1)

one_hot_encoder = OneHotEncoder(categories='auto', sparse=False)
label = one_hot_encoder.fit_transform(label.reshape(-1, 1))
label = label.reshape(label.shape[0], label.shape[1], 1 )

In [73]:
X_train, X_test, label_train, label_test = train_test_split(X, label, test_size=0.25)

In [74]:
X_train.shape, label_train.shape, X_test.shape, label_test.shape

((426, 30, 1), (426, 2, 1), (143, 30, 1), (143, 2, 1))

In [75]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
label_train = torch.from_numpy(label_train)
label_test = torch.from_numpy(label_test)

In [85]:
nn = SimpleNN(30, 50, 2)

epochs = 200

train_loss, test_loss = [], []
for epoch in range(epochs):
    
    train_loss.append(np.mean([nn.loss(x, y).item() for x, y in zip(X_train, label_train)]))
    test_loss.append(np.mean([nn.loss(x, y).item() for x, y in zip(X_test, label_test)]))
    for x, y in zip(X_train, label_train):
        nn.sgd_step(x, y, 1e-4)
    
    print('epoch: {}, train loss: {:.5f}, test loss: {:.5f}'.format(epoch, train_loss[-1], test_loss[-1]))
    


RuntimeError: t() expects a 2D tensor, but self is 3D

In [21]:
class SimpleNN:
    
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        self.w_1 = torch.randn(hidden_size, input_size, dtype=torch.float64, requires_grad=True)
        self.w_2 = torch.randn(output_size, hidden_size, dtype=torch.float64, requires_grad=True)
              
        self.cache = {}
    
    def forward(self, x: torch.tensor):
        h_1 = torch.matmul(self.w_1, x)
        z_1 = torch.sigmoid(h_1)
        h_2 = torch.matmul(self.w_2, z_1)
        z_2 = F.log_softmax(h_2, dim=0)
        
        self.cache['z_1'] = z_1
        self.cache['z_2'] = z_2
        return z_2
    
    def loss(self, x: torch.tensor, label: torch.tensor):
        log_pred = self.forward(x)
        return -torch.sum(label * log_pred)
    
    def backward(self, x: torch.tensor, label: torch.tensor):
        self.forward(x)
        
        z_1, z_2 = self.cache['z_1'], self.cache['z_2']

        dh_2 = torch.exp(z_2) - label
        dw_2 = torch.matmul(dh_2, z_1.t())
        dh_1 = torch.matmul(self.w_2.t(), dh_2) * (z_1 * (1 - z_1))
        dw_1 = torch.matmul(dh_1, x.t())
        return dw_1, dw_2 
    
    
    def numerical_gradients(self, x: torch.tensor, label: torch.tensor, epsilon: float):
        d_params = (
            torch.zeros_like(self.w_1, dtype=torch.float64),  
            torch.zeros_like(self.w_2, dtype=torch.float64)
        )
        params = (self.w_1, self.w_2)

        # calculating numerical gradients for each parameter
        for d_param, param in zip(d_params, params):

            # iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
            it = np.nditer(param.detach(), flags=['multi_index'], op_flags=['readwrite'])
            while not it.finished:
                ix = it.multi_index

                # keeping the original value so we can reset it later
                original_value = param[ix].item()

                # estimating numeric gradients

                # x + epsilon
                param[ix] = original_value + epsilon
                loss_plus = self.loss(x, label)

                # x - epsilon
                param[ix] = original_value - epsilon
                loss_minus = self.loss(x, label)

                # numeric_gradient = (f(x + delta) - f(x - delta)) / (2 * delta)
                d_param[ix] = ((loss_plus - loss_minus) / (2 * epsilon)).item()

                # resetting parameter to original value
                param[ix] = original_value
                it.iternext()

        return d_params

        
    def gradient_check(self, 
                       x: torch.tensor,
                       label: torch.tensor,
                       epsilon: float = 1e-1,
                       threshold: float = 1e-5):
        """
        Performs gradient checking for model parameters:
         - computes the analytic gradients using our back-propagation implementation
         - computes the numerical gradients using the two-sided epsilon method
         - computes the relative difference between numerical and analytical gradients
         - checks that the relative difference is less than threshold
         - if the last check is failed, then raises an error
        """
        
        def check_relative_difference(a: torch.tensor, b: torch.tensor, threshold: float) -> bool:
            """Returns True if (|a - b| / (|a| + |b|)) > threshold else False."""
            numeratore = torch.abs(a - b)
            denominatore = torch.abs(a) + torch.abs(b)
            result = numeratore / denominatore
            result[torch.isnan(result)] = 0
            return bool(torch.any(result > threshold))
        
        params = ('w_1', 'w_2')

        # calculating the gradients using backpropagation, aka analytic gradients
        self.cache = {}
        analytic_gradients = self.backward(x, label)

        # calculating numerical gradients
        self.cache = {}
        numeric_gradients = self.numerical_gradients(x, label, epsilon)

        # gradient check for each parameter
        for p_name, d_analytic, d_numeric in zip(params, analytic_gradients, numeric_gradients):
            print(f"\nPerforming gradient check for parameter {p_name} "
                  f"with size = {np.prod(d_analytic.shape)}.")
            
            if (not d_analytic.shape == d_numeric.shape or
                    check_relative_difference(d_analytic, d_numeric, threshold)):
                            
                raise ValueError(f'Gradient check for {p_name} is failed.')

            print(f"Gradient check for parameter {p_name} is passed.")
        

In [38]:
print('Testing implementation.')

nn = SimpleNN(10, 20, 3)
# nn.w_1 = w_1
# nn.w_2 = w_2

x = torch.arange(10, dtype=torch.float64).view(10, 1)
label = torch.tensor([0, 0, 1.], dtype=torch.float64).reshape(3, 1)

log_pred = nn.forward(x)
pred = torch.exp(log_pred)

print(sum(pred))
print(pred)
assert pred.shape == label.shape == (3, 1)
assert bool(abs(sum(pred) - 1.) < 1e-6)

loss = nn.loss(x, label)
print(loss.item())
assert torch.equal(loss, -log_pred[2, 0])

dw_1, dw_2 = nn.backward(x, label)
assert dw_1.shape == nn.w_1.shape == (20, 10)
assert dw_2.shape == nn.w_2.shape == (3, 20)
loss.backward()



# nn.gradient_check(x, label, epsilon=1e-2, threshold=1e-3)

Testing implementation.
tensor([1.0000], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[0.9810],
        [0.0142],
        [0.0048]], dtype=torch.float64, grad_fn=<ExpBackward>)
5.333037773402289


In [45]:
torch.allclose(nn.w_1.grad.data, dw_1, atol=1e-15, rtol=0)

True

In [51]:
torch.allclose(nn.w_2.grad.data, dw_2, atol=1e-15, rtol=0)

True

In [46]:
def check_relative_difference(a: torch.tensor, b: torch.tensor, threshold: float) -> bool:
    """Returns True if (|a - b| / (|a| + |b|)) > threshold else False."""
    numeratore = torch.abs(a - b)
    denominatore = torch.abs(a) + torch.abs(b)
    result = numeratore / denominatore
    result[torch.isnan(result)] = 0
    return bool(torch.any(result > threshold))

In [50]:
check_relative_difference(nn.w_1.grad.data, dw_1, 1e-15)

False

In [52]:
check_relative_difference(nn.w_2.grad.data, dw_2, 1e-15)

False

In [ ]:
list(data.keys())

In [ ]:
data['data'].shape